# OPTUNA 

MIT licence.

3 principaux objectifs :
1. Construction de l'espace de recherche de façon dynamique (*define-by-run API*)
2. Implémentation efficiente de la recherche mais également du *pruning* (c'est-à-dire quand des branches entières de l'espace de recherche sont supprimées)
3. Utilisation simple allant de l'architecture légère à l'architecture extensible et distribuée.

In [ ]:
# imports
# %matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from functools import partial
import warnings
warnings.filterwarnings('ignore')

import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
from sklearn.utils import shuffle
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm
from sklearn import datasets, svm, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import train_test_split

# visualisation
from optuna.visualization.matplotlib import plot_contour
from optuna.visualization.matplotlib import plot_edf
from optuna.visualization.matplotlib import plot_intermediate_values
from optuna.visualization.matplotlib import plot_optimization_history
from optuna.visualization.matplotlib import plot_parallel_coordinate
from optuna.visualization.matplotlib import plot_param_importances
from optuna.visualization.matplotlib import plot_slice 

SEED = 42
np.random.seed(SEED)

In [ ]:
def plot_digits(images, count=4):
    ncols = 4
    nrows = (count + ncols - 1)//ncols
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 3*nrows))
    axes = axes.flatten()
    for ax, image in zip(axes, images):
        ax.set_axis_off()
        ax.imshow(image., cmap=plt.cm.gray_r, interpolation="nearest")
        # ax.set_title("Digit: %i" % label)

In [ ]:
digits = datasets.load_digits() # data images, target, target_names

split_index = 1000
x = digits.data
y = digits.target
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, train_size=0.5, random_state=0)

plot_digits(digits.images)

# Introductory example

As an introductory example, we use a Decision Tree Classifier to predict the digit from the image.

In [ ]:
digits_dt = tree.DecisionTreeClassifier(criterion='entropy', max_depth=20, min_samples_leaf=10)
digits_dt.fit(x_train, y_train)
prediction = digits_dt.predict(x_test)
print("Generalization error:", np.sum(np.not_equal(prediction, y_test))/len(y_test) )
print("Generalization score:", digits_dt.score(x_test, y_test))

Suppose that we want find the best hyperparameters to maximize the score. The most naive way to do it is by grid-search.

In [ ]:
best_score = 0
best_params = None
nb_trials = 0
for depth in [10, 20, 30]:
    for min_samples_leaf in [3, 5, 10, 20]:
        digits_dt = tree.DecisionTreeClassifier(criterion='entropy', max_depth=depth, min_samples_leaf=min_samples_leaf)
        digits_dt.fit(x_train, y_train)
        prediction = digits_dt.predict(x_test)
        score = digits_dt.score(x_test, y_test)
        if(score > best_score):
            best_score = score
            best_params = (depth, min_samples_leaf)
        nb_trials += 1

print('Best score={:0.2f} obtained for parameters={} after {} trials.'.format(score, best_params, nb_trials))

As you see, we have to statically define the parameters to try and somewhat trial-and-error our way to the best solution, even with grid search. Of course, the Design of Experiment (DoE) can be improved quite easily, using latin hypercube for example. 

However, Optuna, an optimization framework, offers the possibility to ease this process in a very different way :
- We define an objective function that calls, for the parameters we want to optimize on, the `suggest_API` and outputs a objevtive value (to minimize in this case).
- Then, we define a study. 
- Then we can simply call the `optimize` function. By default it seeks to minimize the score.

After the trial is over, finding the best set of parameters is done by calling member fields of the study object.

In [ ]:
def objective_(trial, x, y):
    sug_max_depth = trial.suggest_int('rf_max_depth', 2, 32)
    sug_min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 20)

    classifier_obj = sklearn.ensemble.RandomForestClassifier(max_depth=sug_max_depth, min_samples_leaf=sug_min_samples_leaf)
    x_train, x_valid, y_train, y_valid = sklearn.model_selection.train_test_split(x, y, train_size=0.5, random_state=0)
    classifier_obj.fit(x_train, y_train)
    score = classifier_obj.score(x_valid, y_valid)
    return score

n_trials = 10
objective = partial(objective_, x=x_train, y=y_train)
study = optuna.create_study()  # Create a new study.
study.optimize(lambda trial: 1.0 - objective(trial), n_trials=n_trials)  # Invoke optimization of the objective function.
print('Best score={:0.2f} obtained for parameters={} after {} trials.'.format(1-study.best_value, study.best_params, n_trials))

So with the same number of trials, we found a better solution than what we could do with simple grid-search. Optuna also makes it easier to understand what goes on during the study with plots as seen afterwards. For more information, you can check [here](https://optuna.readthedocs.io/en/latest/tutorial/10_key_features/005_visualization.html#visualization). 

In [ ]:
plot_optimization_history(study);
# plot_parallel_coordinate(study);
# plot_param_importances(study);

## Optuna Search algorithm
It is now time to discuss a bit more what goes on in Optunaw with the Suggest_API.

[...]

## What can we really do with Optuna ?

Optuna can suggest three types of features ([source](https://optuna.readthedocs.io/en/latest/tutorial/10_key_features/002_configurations.html#sphx-glr-tutorial-10-key-features-002-configurations-py)):
- categorical : `trial.suggest_categorical(str:name, choices:Sequence[Union[None, bool, int, float, str]])`
- integers : `trial.suggest_int(name:str, low:int, high:int, step=1, log=False)`
- floats : `trial.suggest_float(name:str, low:float, high:float, step=None, log=False)`


The parameters they take in inputs are mostly quite clear :
- *log* : if `True`, the value is sampled uniformely from the range in the log domain.
- *choices : can be a sequence of anything, e.g. ['Random Forest Classifier', 'KNeightbors'] or [None, np.pi, 'SVM', 0, True].
- high is **included**


Next is an exercice. Propose an objective function `objective_ex1` that optimizes, for the NIST dataset, on :
- Random Forest Classifier on the maximum depth and minimum samples in leaf
- Gaussian Process Classifier on kernel (try for example the Radial Basis `RBF` with different value for the length scale)
- Support Vector Classifier on the Kernel coefficient and Regularization parameter

Run the study for 10 seconds (using the `timeout` parameter in the `study.optimize` function).

# TODO : 
Ajouter la définition de ces fonctions dans des exercices avant celui-ci.

In [ ]:
def train_sklearn(trial, obj, x, y):
    x_train, x_valid, y_train, y_valid = sklearn.model_selection.train_test_split(x, y, train_size=0.5, random_state=0)
    obj.fit(x_train, y_train)
    score = obj.score(x_valid, y_valid)
    return score

def svc_objective(trial, x, y):
    sug_C = trial.suggest_float('C', 1e-10, 1e10, log=True)
    sug_gamma_kind = trial.suggest_categorical('gamma_kind', ['auto', 'scale', 'float'])
    if(sug_gamma_kind == 'float'):
        sug_gamma = trial.suggest_float('gamma', 1e-3, 10., log=True)
    else:
        sug_gamma = sug_gamma_kind
    classifier_obj = sklearn.svm.SVC(C=sug_C, gamma=sug_gamma)
    return train_sklearn(trial, classifier_obj, x, y)

def random_forest_classifier_objective(trial, x, y):
    sug_max_depth = trial.suggest_int('max_depth', 2, 32)
    sug_min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 20)
    classifier_obj = sklearn.ensemble.RandomForestClassifier(max_depth=sug_max_depth, min_samples_leaf=sug_min_samples_leaf)
    return train_sklearn(trial, classifier_obj, x, y)

def gaussian_Process_classifier_objective(trial, x, y):
    sug_length_scale = trial.suggest_float('length_scale', 1e-3, 10, log=True)
    kernel = RBF(length_scale=sug_length_scale)
    classifier_obj = GaussianProcessClassifier(kernel=kernel)
    return train_sklearn(trial, classifier_obj, x, y)

def objective_ex1(trial, x, y):
    classifier_name = trial.suggest_categorical('classifier', ['SVC', 'RandomForestClassifier', 'GaussianProcessClassifier'])
    if(classifier_name == 'SVC'):
        score = svc_objective(trial, x, y)
    elif(classifier_name  == 'RandomForestClassifier'):
        score = random_forest_classifier_objective(trial, x, y)
    elif(classifier_name  == 'GaussianProcessClassifier'):
        score = gaussian_Process_classifier_objective(trial, x, y)
    return score


In [ ]:
timeout = 10
objective = partial(objective_ex1, x=x_train, y=y_train)
study = optuna.create_study()  # Create a new study.
study.optimize(lambda trial: 1.0 - objective(trial), timeout=timeout)  # Invoke optimization of the objective function.
print('Best score={:0.5f} obtained for parameters={} after {} trials.'.format(1-study.best_value, study.best_params, n_trials))

In [ ]:
# study.trials_dataframe()

So we see that we, very quickly, can get a very good classifier and quite easily in fact. But, that is not all that Optuna can do. There is also an overhead when creating the study object, as can be seen from the total duration compared to the choose timeout ! 

Indeed, Optuna has a pruning algorithm that allows it to cut short specific trials if it seens that intermediate results are not going so well. When done smartly, this allows to cut entire region of the parameter search space and thus explore much more quickly possibilities. It is to be noted that it can only be used when one can compute intermediate results.

When dealing with the images dataset, it can make sense to use Deep Learning and more specifically, Convolutional Neural Network.

More information [here](https://optuna.readthedocs.io/en/latest/tutorial/10_key_features/003_efficient_optimization_algorithms.html).

# Uping our game : MNIST
Let's try on MNIST.

In [ ]:
import torch
from torchvision import datasets as tdatasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data = tdatasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)
test_data = tdatasets.MNIST(
    root = 'data', 
    train = False,
    transform = ToTensor()
)

# Limiting the size of the training set because it takes too much time otherwise !
limit_train = 1000

# subset training set
index_sub = np.random.choice(np.arange(len(train_data)), limit_train, replace=False)

#replacing attribute
train_data.data = train_data.data[index_sub]
train_data.targets = train_data.targets[index_sub]

# train_data, valid_data = torch.utils.data.random_split(train_data_full, [limit_train, len(train_data_full) - limit_train])

In [ ]:
plot_digits(train_data.data[:4].numpy())

In [ ]:
x_train_, y_train_ = train_data.data[:limit_train].numpy().reshape(limit_train, -1), train_data.targets[:limit_train].numpy()

In [ ]:
timeout = 10
objective = partial(objective_ex1, x=x_train_, y=y_train_)
study = optuna.create_study()  # Create a new study.
study.optimize(lambda trial: 1.0 - objective(trial), timeout=timeout)  # Invoke optimization of the objective function.
print('Best score={:0.5f} obtained for parameters={} after {} trials.'.format(1-study.best_value, study.best_params, n_trials))

So we see that we don't succeed anymore in reaching top score. Let's now try with deep learning and convolutional neural networks...

In [ ]:
def get_h_out(h_in, kernel_size, padding=0, dilation=1, stride=1):
    return np.floor((h_in + 2 * padding - dilation * (kernel_size - 1) -1)/stride + 1)

def get_w_out(w_in, kernel_size, padding=0, dilation=1, stride=1):
    return np.floor((w_in + 2 * padding - dilation * (kernel_size - 1) -1)/stride + 1)

class SmallConvNet(torch.nn.Module):
    def __init__(self, c_in_list, c_out_list, kernel_size_list, output_size):
        super(SmallConvNet, self).__init__()
        conv_list = []
        assert((len(c_in_list) == len(c_out_list)) and (len(kernel_size_list) == len(c_out_list)))
        for c_in, c_out, k in zip(c_in_list, c_out_list, kernel_size_list):
            conv_list.append(torch.nn.Conv2d(c_in, c_out, k))
        self.conv = torch.nn.Sequential(*conv_list)
        self.fc1 = torch.nn.Linear(output_size, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv(x)
        x = torch.nn.functional.relu(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = self.fc2(x)
        output = torch.nn.functional.softmax(x, dim=1)
        return output

class TorchClassifier:
    def __init__(self, model, lr, verbose=True) -> None:
        self.model = model
        self.lr = lr
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self.model.to(self.device)
        self.loss_function = torch.nn.CrossEntropyLoss()
        self.optimiser = torch.optim.SGD(self.model.parameters(), lr=lr, momentum=0.9)
        self.loss_function = torch.nn.CrossEntropyLoss()
        self.verbose = verbose

    def train(self, train_loader, epochs):
        train_loss_list = []
        for epoch in tqdm(range(epochs), disable=not self.verbose):
            train_loss = self.partial_fit(epoch, train_loader)
            train_loss_list.append(train_loss)
        return train_loss_list

    def train_one_epoch(self, train_loader):
        loss_list = []
        for x, y in train_loader:
            pred = self.model(x)
            self.optimiser.zero_grad()
            loss = self.loss_function(pred, y)
            loss.backward()
            self.optimiser.step()
            loss_list.append(loss.detach().cpu().numpy())
        return np.mean(loss_list)

    def predict(self, test_loader, return_true=False):
        self.model.eval()
        pred_list = []
        true_list = []
        with torch.no_grad():
            for x, y in test_loader:
                pred = self.model(x)
                pred_list.append(pred)
                true_list.append(y)
        self.model.train()
        if(return_true):
            return torch.concatenate(true_list, axis=0), torch.concatenate(pred_list, axis=0)
        return torch.concatenate(pred_list, axis=0)
    
    def score(self, test_loader):
        true, pred = self.predict(test_loader, return_true=True)
        accuracy = get_accuracy(true.cpu().numpy(), pred.cpu().numpy())
        return accuracy

def get_accuracy(y_true, y_prob):
    assert (y_true.ndim == 1 and y_true.shape[0] == y_prob.shape[0])
    y_prob = np.argmax(y_prob, axis=-1)
    return 1 - np.mean(np.abs(y_true - y_prob))


def get_valid_predictions(net, validset):
    validloader = torch.utils.data.DataLoader(validset, batch_size=4, shuffle=False)
    all_labels = np.array([])
    predictions = np.array([])
    with torch.no_grad():
        for data in validloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            all_labels = np.append(all_labels, labels.numpy())
            predictions = np.append(predictions, predicted.numpy())
    return all_labels, predictions

With that, write an objective function, called `cnn_naive_objective`.

In [ ]:
def train_torch(trial, obj, train_data, valid_data):
    train_loader = torch.utils.data.DataLoader(train_data, 
                                              batch_size=16, 
                                              shuffle=True, 
                                              num_workers=1)
    valid_loader = torch.utils.data.DataLoader(valid_data, 
                                               batch_size=16, 
                                               shuffle=True, 
                                               num_workers=1)
    for step in range(3):
        obj.train_one_epoch(train_loader)
        intermediate_value = obj.score(valid_loader)
            
        trial.report(intermediate_value, step)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.TrialPruned()

    return 1.0 - obj.score(valid_loader)

def cnn_naive_objective(trial, train_data, valid_data):
    n_layers = trial.suggest_int('n_layers', 1, 3)
    c_in_list = [1]
    c_out_list = []
    k_list = []
    h = train_data.data[0].shape[0]
    w = train_data.data[0].shape[1]
    for n in range(n_layers):
        if(n != 0):
            c_in_list.append(c_out_list[-1])
        c_out_list.append(trial.suggest_int(f'c_out_{n}', 1, 4))
        k_list.append(trial.suggest_int(f'k_{n}', 3, 7, 2))
        w = get_w_out(w, k_list[-1])
        h = get_h_out(h, k_list[-1])

    convnet = SmallConvNet(
        c_in_list=c_in_list,
        c_out_list=c_out_list,
        kernel_size_list=k_list,
        output_size=int(h*w*c_out_list[-1])
    )

    sug_lr = trial.suggest_float('lr', 1e-5, 1, log=True)
    classifier = TorchClassifier(
        convnet,
        lr=sug_lr
    )
    
    return train_torch(trial, classifier, train_data, valid_data)

In [ ]:
timeout = 120
objective = partial(cnn_naive_objective, train_data=train_data, valid_data=test_data)
study = optuna.create_study(pruner=optuna.pruners.MedianPruner())
study.optimize(lambda trial: 1.0 - objective(trial), timeout=timeout, show_progress_bar=True)  # Invoke optimization of the objective function.
print('Best score={:0.5f} obtained for parameters={} after {} trials.'.format(1-study.best_value, study.best_params, n_trials))